In [1]:
import numpy as np
import pandas as pd
import lxml.etree as etree
import sqlite3 as sql
from process import DB
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [6]:
data = pd.read_csv('../data/prem_data_elo.csv', index_col = 'Unnamed: 0')

In [17]:
X = data[data.columns[7:]].drop(['result', 'home_team_api_id_y', 'away_team_api_id_y'], axis = 1)
y = data['result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [18]:
def accuracy(clf, X, y):
    return np.mean(clf.predict(X) == y)

In [20]:
params = {
    'multi_class' : 'multinomial'
}
clf = LogisticRegression(**params).fit(X_train, y_train)

print(accuracy(clf, X_train, y_train))
print(accuracy(clf, X_test, y_test))

0.8968668407310705
0.8991304347826087


/Users/kayacelebi/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8991304347826087